In [89]:
import numpy as np

if not hasattr(np, "bool8"):

    np.bool8 = np.bool_

In [90]:
from shimmy import GymV21CompatibilityV0
import stable_baselines3 as sb3
import matplotlib.pyplot as plt
import gymnasium
import gym
import crafter
import wandb
import cv2

In [91]:
def image_display(img):
    
    plt.figure(figsize = (6, 6))
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    plt.axis('off')
    
    plt.show()

In [92]:
def init_env():
    
    gym.envs.registration.register(id = 'CrafterNoReward-v1', entry_point = crafter.Env)
    
    env = gym.make('CrafterNoReward-v1')

    if not hasattr(env, "seed"):

        env.seed = lambda seed = None: None
    
    env = crafter.Recorder(
        env, './path/to/logdir',
        save_stats=True,
        save_video=False,
        save_episode=False,
    )
    
    env = GymV21CompatibilityV0(env = env)
    
    return env 

In [114]:
def evaluation_episode(env, model, config):
    
    wandb.init(entity = "reinforcement-learning-wits", project = "Crafter", name = "", config = config) 
    
    observation, info = env.reset() 
    
    terminated = False
    
    reward_sum = 0
    
    while not terminated:
        
        action, _ = model.predict(observation, deterministic = True)
        
        observation, reward, terminated, truncated, info = env.step(action)
        
        info['achievements']['reward'] = reward
        
        wandb.log(info['achievements'])
    
    wandb.finish()

In [115]:
env_vec = sb3.common.env_util.make_vec_env(env_id = init_env, n_envs = 8)

/home/vmuser/anaconda3/envs/crafter_env/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment CrafterNoReward-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [116]:
model = PPO("CnnPolicy", env_vec, learning_rate = 2.5e-4, n_steps = 256, batch_size = 64, verbose = 1)

model.learn(total_timesteps = 10000)

Using cpu device
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 167      |
|    ep_rew_mean     | 1.35     |
| time/              |          |
|    fps             | 106      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 176        |
|    ep_rew_mean          | 1.1        |
| time/                   |            |
|    fps                  | 114        |
|    iterations           | 2          |
|    time_elapsed         | 35         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01408878 |
|    clip_fraction        | 0.131      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.82      |
|    explained_variance   | 

In [117]:
e = init_env()
evaluation_episode(e, model, {})

/home/vmuser/anaconda3/envs/crafter_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/vmuser/anaconda3/envs/crafter_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/vmuser/anaconda3/envs/crafter_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/home/vmuser/anaconda3/envs/crafter_env/lib/python3.10/site-

collect_coal,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
collect_diamond,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
collect_drink,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
collect_iron,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
collect_sapling,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████
collect_stone,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
collect_wood,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
defeat_skeleton,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
defeat_zombie,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eat_cow,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+13,...
